**GOAL**: predict the number of violent crimes

**DATA**: the UCI communities dataset, contains demographic information about counties in the US

**Import modules**

In [37]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statistics import mean
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

**Import datasets**

In [2]:
data = pd.read_csv(r"C:\Users\BrechtDewilde\Documents\UGENT -  statistical data analysis\STATISTICAL DATA ANALYSIS\SEM 2\Big data science\datasets\communities.psv", sep = "|")

In [3]:
# The features are named A0 - A112 (more information about the features can be found on the UCI site)
data.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A113,A114,A115,A116,A117,A118,A119,A120,A121,A122
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


**Data Cleaning**

* missing data indicated as a question mark
* split the data into x and y whereby the dependent attribute is the last (A122)
* missing values are replaced by column means 

In [4]:
data = pd.read_csv(r"C:\Users\BrechtDewilde\Documents\UGENT -  statistical data analysis\STATISTICAL DATA ANALYSIS\SEM 2\Big data science\datasets\communities.psv", sep = "|", na_values = "?")
data = data.fillna(data.mean())

In [5]:
x = data.loc[:,data.columns != "A122"]
y = data.loc[:,data.columns[-1]]

**Linear Regression with cross-validation**

In [6]:
# Create the model 
lr = LinearRegression(normalize = True)

# Create the fold class
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

# list containing the fold metrics
cv_scores = []

# fit a model for each fold
for train_index, test_index in kf.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)
    cv_scores.append(mean_squared_error(y_test,y_pred))

In [7]:
round(mean(cv_scores), 3)

0.093

**Feature Selection**
There exist several strategies for feature selection.

The RFECV class from scikit learn automates backward feature selection.

In [8]:
from sklearn.feature_selection import RFECV
estimator = LinearRegression(normalize=True)
selector = RFECV(estimator, 1, cv = 5)
selector = selector.fit(x,y)

In [9]:
print("The best amount of features to select: {}".format(selector.n_features_))

The best amount of features to select: 14


In [10]:
print("The best feature subset:")
x.columns[selector.support_]

The best feature subset:


Index(['A0', 'A10', 'A20', 'A21', 'A38', 'A40', 'A41', 'A44', 'A64', 'A71',
       'A79', 'A80', 'A97', 'A103'],
      dtype='object')

In [11]:
print("The ranking of the features is as follows")
x.columns[selector.ranking_][:15]

The ranking of the features is as follows


Index(['A1', 'A99', 'A8', 'A84', 'A86', 'A45', 'A50', 'A30', 'A61', 'A82',
       'A1', 'A62', 'A28', 'A24', 'A59'],
      dtype='object')

**Linear regression with feature selection**

In [12]:
# Create the model 
lr = LinearRegression(normalize = True)

# Create the fold class
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

# list containing the fold metrics
cv_scores = []

# select feature subset
x_fs = x.loc[:,selector.support_]

# fit a model for each fold
for train_index, test_index in kf.split(x_fs):
    x_train, x_test = x_fs.iloc[train_index], x_fs.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)
    cv_scores.append(mean_squared_error(y_test,y_pred))

In [13]:
round(mean(cv_scores), 3)

0.053

**Comparison of performance after feature selection**

The performance is clearly better after feature selection. <br/> In case all features are used, then a MSE of 0.09 is obtained. <br/>With feature selection we have a MSE of 0.053.

### lasso, Ridge and elastic net regression 

Another way to deal with a large number of features is to penalise overfitting using any of these type of regression techniques. However, these techniques use a hyperparameter. 

In [14]:
y = y.to_frame()
# obtain a train and test set, each method will be compared on the same set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0, test_size = 0.2)

***Lasso***

In [49]:
clf = LassoCV(cv = 5, random_state = 0).fit(x_train,y_train)
y_pred = clf.predict(x_test)

C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1100: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
lasso_mse = round(mean_squared_error(y_test, y_pred),2)

In [51]:
lasso_len = len(x_train.columns[abs(clf.coef_) == 0])
lasso_params = x_train.columns[abs(clf.coef_) == 0]

***Ridge***

In [52]:
clf = RidgeCV(cv = 5).fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [53]:
ridge_mse = round(mean_squared_error(y_test, y_pred),2)

**Elastic Net**

In [54]:
regr = ElasticNetCV(cv=5, random_state=0).fit(x_train,y_train)
y_pred = regr.predict(x_test)

C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1100: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [55]:
en = round(mean_squared_error(y_test, y_pred),2)

In [56]:
en_params = x_train.columns[abs(regr.coef_) == 0]
en_len = len(x_train.columns[abs(regr.coef_) == 0])

### Comparison of the different techniques

In [59]:
print("Ridge regression has a MSE: {}".format(ridge_mse))
print("Lasso has a MSE: {}, and a total number of selected features of {}".format(lasso_mse, lasso_len))
print("Elastic net has a MSE: {} and a total number of selected features of {}".format(en, en_len))

Ridge regression has a MSE: 0.02
Lasso has a MSE: 0.02, and a total number of selected features of 78
Elastic net has a MSE: 0.02 and a total number of selected features of 76


In [60]:
print("The features that lasso has selected are: ")
lasso_params

The features that lasso has selected are: 
The features that elastic net has selected are:


Index(['A0', 'A1', 'A4', 'A6', 'A8', 'A10', 'A12', 'A13', 'A14', 'A19', 'A20',
       'A21', 'A26', 'A27', 'A29', 'A30', 'A31', 'A32', 'A33', 'A35', 'A37',
       'A39', 'A40', 'A41', 'A42', 'A47', 'A49', 'A51', 'A52', 'A53', 'A54',
       'A55', 'A56', 'A57', 'A59', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65',
       'A66', 'A67', 'A70', 'A73', 'A77', 'A79', 'A80', 'A81', 'A83', 'A84',
       'A87', 'A89', 'A91', 'A92', 'A93', 'A95', 'A96', 'A97', 'A98', 'A99',
       'A100', 'A102', 'A103', 'A106', 'A107', 'A109', 'A110', 'A111', 'A112',
       'A113', 'A114', 'A115', 'A116', 'A117', 'A121'],
      dtype='object')

In [61]:
print("The features that elastic net has selected are:")
en_params

The features that elastic net has selected are:


Index(['A0', 'A1', 'A4', 'A6', 'A8', 'A10', 'A12', 'A13', 'A14', 'A19', 'A20',
       'A21', 'A26', 'A27', 'A29', 'A30', 'A31', 'A32', 'A33', 'A35', 'A37',
       'A39', 'A40', 'A41', 'A42', 'A47', 'A49', 'A51', 'A52', 'A53', 'A54',
       'A55', 'A56', 'A57', 'A59', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65',
       'A66', 'A67', 'A70', 'A73', 'A77', 'A79', 'A80', 'A81', 'A83', 'A84',
       'A87', 'A89', 'A91', 'A92', 'A93', 'A95', 'A96', 'A97', 'A98', 'A99',
       'A100', 'A102', 'A103', 'A106', 'A107', 'A109', 'A110', 'A111', 'A112',
       'A113', 'A114', 'A115', 'A116', 'A117', 'A121'],
      dtype='object')

In [65]:
lst3 = [feature for feature in en_params if feature in lasso_params] 
lst3

['A0',
 'A1',
 'A4',
 'A6',
 'A8',
 'A10',
 'A12',
 'A13',
 'A14',
 'A19',
 'A20',
 'A21',
 'A26',
 'A27',
 'A29',
 'A30',
 'A31',
 'A32',
 'A33',
 'A35',
 'A37',
 'A39',
 'A40',
 'A41',
 'A42',
 'A47',
 'A49',
 'A51',
 'A52',
 'A53',
 'A54',
 'A55',
 'A56',
 'A57',
 'A59',
 'A60',
 'A61',
 'A62',
 'A63',
 'A64',
 'A65',
 'A66',
 'A67',
 'A70',
 'A73',
 'A77',
 'A79',
 'A80',
 'A81',
 'A83',
 'A84',
 'A87',
 'A89',
 'A91',
 'A92',
 'A93',
 'A95',
 'A96',
 'A97',
 'A98',
 'A99',
 'A100',
 'A102',
 'A103',
 'A106',
 'A107',
 'A109',
 'A110',
 'A111',
 'A112',
 'A113',
 'A114',
 'A115',
 'A116',
 'A117',
 'A121']